In [2]:
import openpyxl
import pandas as pd
import numpy as np
from numpy.linalg import norm

feature_names = ['Length',	'Diameter',	'Height',	'Whole_weight',	'Shucked_weight',	'Viscera_weight',	'Shell_weight',	'Rings']
noOfIterations = 20

def convertSign(value):
    if value == "positive":
        return 1
    elif value == "negative":
        return 0
    else:
        return -1

def generateSigns(feature_names):
    dataframe = openpyxl.load_workbook("outputAbaloneKernelShapLOOCV_1.xlsx")
    sheet = dataframe.active
   
    list = []
    for i in range(len(feature_names)):
        list_feature = []
        list.append(list_feature)
    
    df_KernelShap = pd.DataFrame()
    
    for i, row in enumerate(sheet):
        step = 6
        if i == 0: 
            continue
        for k in range(len(feature_names)):
            if k == 0:
                list[k].append(convertSign(row[3].value))
            else:
                list[k].append(convertSign(row[step].value))
                step = step + 3
    dict = {}
    for i in range(len(feature_names)):
        new_data = {feature_names[i]: list[i]} 
        dict.update(new_data)
        
    df_KernelShap = pd.DataFrame(dict)
     
    df_KernelShap.to_excel("signsAbaloneKernelShap.xlsx")
   
    #################################################################
    dataframe = openpyxl.load_workbook("outputAbaloneTreeShapLOOCV_1.xlsx")
    sheet = dataframe.active
   
    list = []
    for i in range(len(feature_names)):
        list_feature = []
        list.append(list_feature)
    
    df_TreeShap = pd.DataFrame()
    
    for i, row in enumerate(sheet):
        step = 6
        if i == 0: 
            continue
        for k in range(len(feature_names)):
            if k == 0:
                list[k].append(convertSign(row[3].value))
            else:
                list[k].append(convertSign(row[step].value))
                step = step + 3          
          
    dict = {}
    for i in range(len(feature_names)):
        new_data = {feature_names[i]: list[i]} 
        dict.update(new_data)
        
    df_TreeShap = pd.DataFrame(dict)
    
    df_TreeShap.to_excel("signsAbaloneTreeShap.xlsx")    

    ############################################################################
    dataframe = openpyxl.load_workbook("outputAbaloneLIMELOOCV_20.xlsx")
    sheet = dataframe.active
    
    list = []
    for i in range(len(feature_names)):
        list_feature = []
        list.append(list_feature)
    
    df_LIME = pd.DataFrame()
    
    for i, row in enumerate(sheet):
        step = 10
        if i == 0: 
            continue
        for k in range(len(feature_names)):
            if k == 0:
                list[k].append(convertSign(row[5].value))
            else:
                list[k].append(convertSign(row[step].value))
                step = step + 5            
            
    dict = {}
    for i in range(len(feature_names)):
        new_data = {feature_names[i]: list[i]} 
        dict.update(new_data)
        
    df_LIME = pd.DataFrame(dict)
   
    df_LIME.to_excel("signsAbaloneLIME.xlsx")
   

def stackRowsColumns(M, noOfRows, noOfColumns):
    pos = [0] * noOfRows
    for i in range (0, noOfRows):
        pos[i] = i
    
    for i in range(1, noOfRows):
        wsim = [0] * noOfRows
        for j in range(i, noOfRows):
            for k in range(0, i):
                A = M.loc[k]
                B = M.loc[j]
                wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
        max_value = max(wsim)
        max_index = wsim.index(max_value)
        aux = M.loc[max_index]
        M.loc[max_index] = M.loc[i]
        M.loc[i] = aux
        aux = pos[max_index]
        pos[max_index] = pos[i]
        pos[i] = aux
    
    for i in range(1, noOfColumns):
        wsim = [0] * noOfColumns
        for j in range(i, noOfColumns):
            for k in range(0, i):
                A = M.loc[k]
                B = M.loc[j]
                wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
        max_value = max(wsim)
        max_index = wsim.index(max_value)
        aux = M.loc[max_index]
        M.loc[max_index] = M.loc[i]
        M.loc[i] = aux
    return pos, M

def cosineSimilarity(A, B):
    return np.dot(A,B)/(norm(A)*norm(B))

def similarityCase(M, i):
    s = 0
    k = i - 1
    for j in range (1, k + 1):
        s = s + cosineSimilarity(M.loc[i], M.loc[i - j]) * (1 / j)
    return s

def similarityMatrix(M, noOfRows):
    s = 0
    for i in range(1, noOfRows):
        s = s + similarityCase(M, i)
    s = s / (noOfRows - 1)
    return s

def GlobalAlign(MSP, MS, noOfRows):
    if similarityMatrix(MSP, noOfRows) >  similarityMatrix(MS, noOfRows):
        max = similarityMatrix(MSP, noOfRows)
        min = similarityMatrix(MS, noOfRows)
    else:
        max = similarityMatrix(MS, noOfRows)
        min = similarityMatrix(MSP, noOfRows)
    return min / max
    #return similarityMatrix(MSP, noOfRows) / similarityMatrix(MS, noOfRows)


def createMSP(MP, MS, noOfRows):
    MS_copy = MS.copy(deep = True)
    MP_copy = MP.copy(deep = True)
    posS, MS_copy = stackRowsColumns(MS_copy, noOfRows, len(feature_names))
    posP, MP_copy = stackRowsColumns(MP_copy, noOfRows, len(feature_names))
    # MSP = pd.DataFrame(columns = feature_names)
    # for i in range(0, noOfRows):
    #     MSP.loc[posP[i]] = MS_copy.loc[posS[i]]
    return GlobalAlign(MP_copy, MS_copy, noOfRows)

generateSigns(feature_names)
df_KernelShap = pd.read_excel('signsAbaloneKernelShap.xlsx')
df_KernelShap = df_KernelShap[df_KernelShap.columns[1:]]
df_TreeShap = pd.read_excel('signsAbaloneTreeShap.xlsx')
df_TreeShap = df_TreeShap[df_TreeShap.columns[1:]]
df_LIME = pd.read_excel('signsAbaloneLIME.xlsx')
df_LIME = df_LIME[df_LIME.columns[1:]]

results = pd.DataFrame(columns = ['LL', 'LS', 'SL', 'SS'])
noOfInstances = df_KernelShap.shape[0]

for instance in range(0, noOfInstances):
    result = []
    data_Shap = []
    for i in range(int(noOfIterations / 2)):
        data_Shap.append(df_KernelShap.loc[instance].values.flatten().tolist())
        data_Shap.append(df_TreeShap.loc[instance].values.flatten().tolist())
    M_Shap  = pd.DataFrame(data_Shap, columns = feature_names)
    
    data_LIME = []
    for i in range(noOfIterations):
        data_LIME.append(df_LIME.loc[instance * noOfIterations + i].values.flatten().tolist())
    M_LIME  = pd.DataFrame(data_LIME, columns = feature_names)
    
    MP = M_LIME
    MS = M_LIME
    result.append(createMSP(MP, MS, noOfIterations))
    MP = M_LIME
    MS = M_Shap
    result.append(createMSP(MP, MS, noOfIterations))
    MP = M_Shap
    MS = M_LIME
    result.append(createMSP(MP, MS, noOfIterations))
    MP = M_Shap
    MS = M_Shap
    result.append(createMSP(MP, MS, noOfIterations))
    print(result)
    results.loc[instance] = result


results.to_excel("resultsSignsAbaloneMPMS_MinMax.xlsx")

[1.0, 0.7641278720830924, 0.7641278720830924, 1.0]
[1.0, 0.8868436699381488, 0.8868436699381488, 1.0]
[1.0, 0.9624642208239464, 0.9624642208239464, 1.0]
[1.0, 0.7919475343581621, 0.7919475343581621, 1.0]
[1.0, 0.6834776689731233, 0.6834776689731233, 1.0]
[1.0, 0.6643304887660951, 0.6643304887660951, 1.0]
[1.0, 0.776587181089006, 0.776587181089006, 1.0]
[1.0, 0.9475316172954691, 0.9475316172954691, 1.0]
[1.0, 0.7595811351143695, 0.7595811351143695, 1.0]
[1.0, 0.8535327074993129, 0.8535327074993129, 1.0]
[1.0, 0.7839917399755504, 0.7839917399755504, 1.0]
[1.0, 0.9444790806535509, 0.9444790806535509, 1.0]
[1.0, 0.8959444300647221, 0.8959444300647221, 1.0]
[1.0, 0.831884090885412, 0.831884090885412, 1.0]
[1.0, 0.8151339189005214, 0.8151339189005214, 1.0]
[1.0, 0.9805787715591288, 0.9805787715591288, 1.0]
[1.0, 0.8073441721157564, 0.8073441721157564, 1.0]
[1.0, 0.5840364138584266, 0.5840364138584266, 1.0]
[1.0, 0.8029464178875348, 0.8029464178875348, 1.0]
[1.0, 0.733989810840775, 0.73398981

C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_17780\4236570583.py:121: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_17780\4236570583.py:146: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))


[nan, nan, nan, 1.0]
[1.0, 0.5477826229273113, 0.5477826229273113, 1.0]
[1.0, 0.9035328880823895, 0.9035328880823895, 1.0]
[1.0, 0.5633164001544295, 0.5633164001544295, 1.0]
[1.0, 0.9467749242079576, 0.9467749242079576, 1.0]
[1.0, 0.815380187135995, 0.815380187135995, 1.0]
[1.0, 0.5060926303307495, 0.5060926303307495, 1.0]
[1.0, 0.9918573892467499, 0.9918573892467499, 1.0]
[1.0, 0.9595557602894835, 0.9595557602894835, 1.0]
[1.0, 0.5249139499358477, 0.5249139499358477, 1.0]
[1.0, 0.623868031614306, 0.623868031614306, 1.0]
[1.0, 0.9258660459771539, 0.9258660459771539, 1.0]
[1.0, 0.6246102203382912, 0.6246102203382912, 1.0]
[1.0, 0.7355492641615546, 0.7355492641615546, 1.0]
[1.0, 0.615231609286989, 0.615231609286989, 1.0]
[1.0, 0.8578262766659921, 0.8578262766659921, 1.0]
[1.0, 0.761056139731699, 0.761056139731699, 1.0]
[1.0, 0.7714489595467365, 0.7714489595467365, 1.0]
[1.0, 0.793594097500149, 0.793594097500149, 1.0]
[1.0, 0.6695174351590274, 0.6695174351590274, 1.0]
[1.0, 0.696118768847

C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_17780\4236570583.py:121: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_17780\4236570583.py:137: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_17780\4236570583.py:146: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))


[1.0, nan, nan, nan]
[1.0, 0.5050468251570234, 0.5050468251570234, 1.0]
[1.0, 0.7293127828590268, 0.7293127828590268, 1.0]
[1.0, 0.972665986692622, 0.972665986692622, 1.0]
[1.0, 0.9876550480788405, 0.9876550480788405, 1.0]
[1.0, 0.7986831283043311, 0.7986831283043311, 1.0]
[1.0, 0.7355838225074975, 0.7355838225074975, 1.0]
[1.0, 0.857107822866391, 0.857107822866391, 1.0]
[1.0, 0.7339755253507375, 0.7339755253507375, 1.0]
[1.0, 0.6891977922964845, 0.6891977922964845, 1.0]
[1.0, 0.9217572413473568, 0.9217572413473568, 1.0]
[1.0, 0.6409141102000285, 0.6409141102000285, 1.0]
[1.0, 0.7106004921335085, 0.7106004921335085, 1.0]
[1.0, 0.8748750433243173, 0.8748750433243173, 1.0]
[1.0, 0.9351901993960886, 0.9351901993960886, 1.0]
[1.0, 0.6566131946156821, 0.6566131946156821, 1.0]
[1.0, 0.8601237750305849, 0.8601237750305849, 1.0]
[1.0, 0.6274144176243764, 0.6274144176243764, 1.0]
[1.0, 0.5717664409745904, 0.5717664409745904, 1.0]
[1.0, 0.9219833445027856, 0.9219833445027856, 1.0]
[1.0, 0.968837

...

[1.0, nan, nan, nan]
[1.0, 0.7712082776013037, 0.7712082776013037, 1.0]
[1.0, 0.7048324999449934, 0.7048324999449934, 1.0]
[1.0, 0.6684592288336074, 0.6684592288336074, 1.0]
[1.0, 0.7719988970587769, 0.7719988970587769, 1.0]
[1.0, 0.7505985235882269, 0.7505985235882269, 1.0]
[1.0, 0.6714393216910579, 0.6714393216910579, 1.0]
[1.0, 0.9798671234108752, 0.9798671234108752, 1.0]
[1.0, 0.7104060262946728, 0.7104060262946728, 1.0]
[1.0, 0.5156333960114418, 0.5156333960114418, 1.0]
[1.0, 0.6589323906304456, 0.6589323906304456, 1.0]
[1.0, 0.8066502263292634, 0.8066502263292634, 1.0]
[1.0, 0.743412983883846, 0.743412983883846, 1.0]
[1.0, 0.7423749919086523, 0.7423749919086523, 1.0]
[1.0, 0.7872019436032418, 0.7872019436032418, 1.0]


C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_17780\4236570583.py:121: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_17780\4236570583.py:137: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_17780\4236570583.py:146: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))


[1.0, nan, nan, nan]
[1.0, 0.6654761548621931, 0.6654761548621931, 1.0]
[1.0, 0.5567998510605202, 0.5567998510605202, 1.0]
[1.0, 0.8654712191919758, 0.8654712191919758, 1.0]
[1.0, 0.9150905222523805, 0.9150905222523805, 1.0]
[1.0, 0.9952269946589829, 0.9952269946589829, 1.0]
[1.0, 0.8841854021403682, 0.8841854021403682, 1.0]
[1.0, 0.8780776459040969, 0.8780776459040969, 1.0]
[1.0, 0.9101877092709882, 0.9101877092709882, 1.0]
[1.0, 0.9197272330018899, 0.9197272330018899, 1.0]
[1.0, 0.5518424144200489, 0.5518424144200489, 1.0]
[1.0, 0.5426887041648502, 0.5426887041648502, 1.0]
[1.0, 0.8638537651220914, 0.8638537651220914, 1.0]
[1.0, 0.8276837380744482, 0.8276837380744482, 1.0]
[1.0, 0.5142397265837658, 0.5142397265837658, 1.0]
[1.0, 0.5148556046795982, 0.5148556046795982, 1.0]
[1.0, 0.5264104113870713, 0.5264104113870713, 1.0]
[1.0, 0.592505580502404, 0.592505580502404, 1.0]
[1.0, 0.7291782642389587, 0.7291782642389587, 1.0]
[1.0, 0.9461668296047784, 0.9461668296047784, 1.0]
[1.0, 0.5170

C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_17780\4236570583.py:121: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_17780\4236570583.py:137: RuntimeWarning: invalid value encountered in scalar divide
  wsim[j] = wsim[j] + (1 / ( i - k)) * np.dot(A,B)/(norm(A)*norm(B))
C:\Users\STUDENT018\AppData\Local\Temp\ipykernel_17780\4236570583.py:146: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(A,B)/(norm(A)*norm(B))


[1.0, nan, nan, nan]
[1.0, 0.5493579599581419, 0.5493579599581419, 1.0]
